In [1]:
# Import the required libraries and dependencies
import requests
import pandas as pd
import talib
import requests
import numpy as np

from pathlib import Path
from itertools import compress
from matplotlib import pyplot as plt
from mplfinance.original_flavor import candlestick_ohlc

#### Download BTC data and create BTC dataframe.

In [2]:
# Load the data into a Pandas DataFrame
df = pd.read_csv(
    Path('../Resources/BTCUSD_1h.csv'),
    index_col="Date")
df.tail()

,Open,Close,High,Low,Volume
Date,,,,,
2021-11-17 13:00:00,60514.231682,60207.000000,60639.0,60207.0,476.771475
2021-11-17 14:00:00,60207.000000,59724.666301,60388.0,59724.0,824.163709
2021-11-17 15:00:00,59724.000000,59708.630836,59859.0,59483.0,500.389504
2021-11-17 16:00:00,59728.000000,60418.000000,60480.0,59491.0,398.163219
2021-11-17 17:00:00,60418.000000,60069.000000,60439.0,60031.0,109.235020


# Identify Candlesticks Pattern
* Based on the TA-Lib module created by Thomas Bulkowsi’s
* Source Code: https://github.com/CanerIrfanoglu/medium/blob/master/candle_stick_recognition/identify_candlestick.py

In [3]:
# extract OHLC 
op = df['Open'].astype(float)
hi = df['High'].astype(float)
lo = df['Low'].astype(float)
cl = df['Close'].astype(float) 


candle_names = talib.get_function_groups()['Pattern Recognition']

# create columns for each candle
for candle in candle_names:
    # below is same as;
    # df["CDL3LINESTRIKE"] = talib.CDL3LINESTRIKE(op, hi, lo, cl)
    df[candle] = getattr(talib, candle)(op, hi, lo, cl)
    
df.head()

,Open,Close,High,Low,Volume,CDL2CROWS,CDL3BLACKCROWS,CDL3INSIDE,CDL3LINESTRIKE,CDL3OUTSIDE,...,CDLSPINNINGTOP,CDLSTALLEDPATTERN,CDLSTICKSANDWICH,CDLTAKURI,CDLTASUKIGAP,CDLTHRUSTING,CDLTRISTAR,CDLUNIQUE3RIVER,CDLUPSIDEGAP2CROWS,CDLXSIDEGAP3METHODS
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-01 07:00:00,966.48,965.81,967.41,965.80,27.489011,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-01-01 08:00:00,965.81,965.46,965.86,962.60,149.617696,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-01-01 09:00:00,965.41,965.98,966.57,965.37,32.288503,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-01-01 10:00:00,965.99,977.01,978.02,965.99,1061.834778,0,0,0,0,100,...,0,0,0,0,0,0,0,0,0,0
2017-01-01 11:00:00,977.71,978.16,978.89,975.05,712.321636,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Assign value 0,1,-1 for No Pattern, Bull Pattern, and Bear Pattern
df['candlestick_pattern'] = np.nan
df['candlestick_index'] = np.nan
for index, row in df.iterrows():

    # bull pattern 100 or 200
    if any(row[candle_names].values > 0):
        pattern = list(compress(row[candle_names].keys(), row[candle_names].values != 0))[0] + '_Bull'
        df.loc[index, 'candlestick_pattern'] = pattern
        df.loc[index, 'candlestick_index'] = 1
    # bear pattern -100 or -200
    elif any(row[candle_names].values < 0):
        pattern = list(compress(row[candle_names].keys(), row[candle_names].values != 0))[0] + '_Bear'
        df.loc[index, 'candlestick_pattern'] = pattern
        df.loc[index, 'candlestick_index'] = -1
    # no pattern
    else:
        df.loc[index, 'candlestick_pattern'] = "NO PATTERN"
        df.loc[index, 'candlestick_index'] = 0

df = df[['Open', 'Close', 'High', 'Low', 'Volume', 'candlestick_pattern', 'candlestick_index']]
df['1h%'] = df['Close'].pct_change().dropna()
df

,Open,Close,High,Low,Volume,candlestick_pattern,candlestick_index,1h%
Date,,,,,,,,
2017-01-01 07:00:00,966.480000,965.810000,967.41,965.80,27.489011,NO PATTERN,0.0,NaN
2017-01-01 08:00:00,965.810000,965.460000,965.86,962.60,149.617696,NO PATTERN,0.0,-0.000362
2017-01-01 09:00:00,965.410000,965.980000,966.57,965.37,32.288503,CDLENGULFING_Bull,1.0,0.000539
2017-01-01 10:00:00,965.990000,977.010000,978.02,965.99,1061.834778,CDL3OUTSIDE_Bull,1.0,0.011418
2017-01-01 11:00:00,977.710000,978.160000,978.89,975.05,712.321636,NO PATTERN,0.0,0.001177
...,...,...,...,...,...,...,...,...
2021-11-17 13:00:00,60514.231682,60207.000000,60639.00,60207.00,476.771475,CDL3OUTSIDE_Bull,1.0,-0.005077
2021-11-17 14:00:00,60207.000000,59724.666301,60388.00,59724.00,824.163709,CDLCLOSINGMARUBOZU_Bear,-1.0,-0.008011
2021-11-17 15:00:00,59724.000000,59708.630836,59859.00,59483.00,500.389504,CDLDOJI_Bull,1.0,-0.000268


### Integrating Technical Indicators
#### Simple Moving Average (SMA)

In [11]:
from ta.trend import SMAIndicator

# Add Simple Moving Average (SMA) indicators
df["sma3h"] = SMAIndicator(close=df["Close"], window=3, fillna=True).sma_indicator()
df["sma24h"] = SMAIndicator(close=df["Close"], window=24, fillna=True).sma_indicator()
df["sma7d"] = SMAIndicator(close=df["Close"], window=168, fillna=True).sma_indicator()
df["sma14d"] = SMAIndicator(close=df["Close"], window=336, fillna=True).sma_indicator()

# Review the DataFrame
df

,Open,Close,High,Low,Volume,candlestick_pattern,candlestick_index,1h%,Short_SMA,Long_SMA,Signal,Entry/Exit,Returns,sma3h,sma24h,sma7d,sma14d
Date,,,,,,,,,,,,,,,,,
2017-01-01 07:00:00,966.480000,965.810000,967.41,965.80,27.489011,NO PATTERN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,965.810000,965.810000,965.810000,965.810000
2017-01-01 08:00:00,965.810000,965.460000,965.86,962.60,149.617696,NO PATTERN,0.0,-0.000362,NaN,NaN,0.0,0.0,0.000000,965.635000,965.635000,965.635000,965.635000
2017-01-01 09:00:00,965.410000,965.980000,966.57,965.37,32.288503,CDLENGULFING_Bull,1.0,0.000539,965.750000,NaN,0.0,0.0,0.000000,965.750000,965.750000,965.750000,965.750000
2017-01-01 10:00:00,965.990000,977.010000,978.02,965.99,1061.834778,CDL3OUTSIDE_Bull,1.0,0.011418,969.483333,NaN,0.0,0.0,0.000000,969.483333,968.565000,968.565000,968.565000
2017-01-01 11:00:00,977.710000,978.160000,978.89,975.05,712.321636,NO PATTERN,0.0,0.001177,973.716667,NaN,0.0,0.0,0.000000,973.716667,970.484000,970.484000,970.484000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-17 13:00:00,60514.231682,60207.000000,60639.00,60207.00,476.771475,CDL3OUTSIDE_Bull,1.0,-0.005077,60505.756998,60005.968040,1.0,0.0,0.000000,60505.756998,60005.968040,63722.955580,63633.695269
2021-11-17 14:00:00,60207.000000,59724.666301,60388.00,59724.00,824.163709,CDLCLOSINGMARUBOZU_Bear,-1.0,-0.008011,60148.632661,59956.287470,1.0,0.0,0.000000,60148.632661,59956.287470,63671.487010,63626.914514
2021-11-17 15:00:00,59724.000000,59708.630836,59859.00,59483.00,500.389504,CDLDOJI_Bull,1.0,-0.000268,59880.099046,59915.355421,0.0,-1.0,-59708.630836,59880.099046,59915.355421,63620.297692,63619.496749


#### Bollinger Bands

In [14]:
from ta.volatility import BollingerBands

# Add Bollinger Bands indicator
indicator_bb = BollingerBands(close=df["Close"], window=20, window_dev=2)
df['bb_bbm'] = indicator_bb.bollinger_mavg()
df['bb_bbh'] = indicator_bb.bollinger_hband()
df['bb_bbl'] = indicator_bb.bollinger_lband()

# Review the DataFrame
df

,Open,Close,High,Low,Volume,candlestick_pattern,candlestick_index,1h%,Short_SMA,Long_SMA,Signal,Entry/Exit,Returns,sma3h,sma24h,sma7d,sma14d,bb_bbm,bb_bbh,bb_bbl
Date,,,,,,,,,,,,,,,,,,,,
2017-01-01 07:00:00,966.480000,965.810000,967.41,965.80,27.489011,NO PATTERN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,965.810000,965.810000,965.810000,965.810000,NaN,NaN,NaN
2017-01-01 08:00:00,965.810000,965.460000,965.86,962.60,149.617696,NO PATTERN,0.0,-0.000362,NaN,NaN,0.0,0.0,0.000000,965.635000,965.635000,965.635000,965.635000,NaN,NaN,NaN
2017-01-01 09:00:00,965.410000,965.980000,966.57,965.37,32.288503,CDLENGULFING_Bull,1.0,0.000539,965.750000,NaN,0.0,0.0,0.000000,965.750000,965.750000,965.750000,965.750000,NaN,NaN,NaN
2017-01-01 10:00:00,965.990000,977.010000,978.02,965.99,1061.834778,CDL3OUTSIDE_Bull,1.0,0.011418,969.483333,NaN,0.0,0.0,0.000000,969.483333,968.565000,968.565000,968.565000,NaN,NaN,NaN
2017-01-01 11:00:00,977.710000,978.160000,978.89,975.05,712.321636,NO PATTERN,0.0,0.001177,973.716667,NaN,0.0,0.0,0.000000,973.716667,970.484000,970.484000,970.484000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-17 13:00:00,60514.231682,60207.000000,60639.00,60207.00,476.771475,CDL3OUTSIDE_Bull,1.0,-0.005077,60505.756998,60005.968040,1.0,0.0,0.000000,60505.756998,60005.968040,63722.955580,63633.695269,59872.711649,60895.619230,58849.804067
2021-11-17 14:00:00,60207.000000,59724.666301,60388.00,59724.00,824.163709,CDLCLOSINGMARUBOZU_Bear,-1.0,-0.008011,60148.632661,59956.287470,1.0,0.0,0.000000,60148.632661,59956.287470,63671.487010,63626.914514,59863.354399,60888.082370,58838.626427
2021-11-17 15:00:00,59724.000000,59708.630836,59859.00,59483.00,500.389504,CDLDOJI_Bull,1.0,-0.000268,59880.099046,59915.355421,0.0,-1.0,-59708.630836,59880.099046,59915.355421,63620.297692,63619.496749,59837.312487,60849.900546,58824.724428
